In [118]:
# vector_db.py
import os
import ast
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from pymilvus import MilvusClient, FieldSchema, CollectionSchema, DataType
from model_config import load_embedding_model_VN

load_dotenv()
MILVUS_ENDPOINT = os.getenv("MILVUS_ENDPOINT")
MILVUS_TOKEN = os.getenv("MILVUS_TOKEN")
from model_config import load_gpt4o_mini_model

llm = load_gpt4o_mini_model()

In [29]:

class MilvusRetrieval:
    def __init__(self, collection_name: str, embedding_model=None):
        """
        Manages vector retrieval from Milvus.

        :param collection_name: Name of the Milvus collection.
        :param embedding_function: Function to convert text queries to embedding vectors.
        """
        self.collection_name = collection_name
        self.client = MilvusClient(uri=MILVUS_ENDPOINT, token=MILVUS_TOKEN)
        self.embedding_model = embedding_model or load_embedding_model_VN()

    def similarity_search(self, query: str, top_k: int = 25):
        """
        Perform a similarity search in Milvus with a query.
        """
        query_vector = self.embedding_model.encode(query)
        search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}
        
        # Perform search in Milvus
        results = self.client.search(
            collection_name=self.collection_name,
            data=[query_vector],
            anns_field="embedding",
            search_params=search_params,
            limit=top_k,
            output_fields=["metadata", "text"]
        )
        return results

In [30]:
retrieval = MilvusRetrieval("test")


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [121]:
query = "các loại học bổng của đại học Bách Khoa Hà Nội"

In [43]:
results = retrieval.similarity_search(query=query, top_k=5)

In [44]:
print(results[0])

[{'id': 453466849343023300, 'distance': 0.6481146216392517, 'entity': {'metadata': {'source': '20230710 1. QĐ Học bổng KKHT 2023.pdf', 'page': 0, 'id': '3'}, 'text': 'Hà Nội . 2. Quy định này áp dụng đối với sinh viên đại học hình thức đào tạo chính quy văn bằng thứ nhất của ĐHBK Hà Nội . Điều 2. Các quy định chung 1. Học bổng khuyến khích học tập (KKHT) cấp cho các sinh viên được lựa chọn bởi Hội đồng xét cấp học bổng'}}, {'id': 453466849343023298, 'distance': 0.6385599970817566, 'entity': {'metadata': {'source': '20230710 1. QĐ Học bổng KKHT 2023.pdf', 'page': 0, 'id': '1'}, 'text': 'và cấp học bổng cho sinh viên đại học hình thức đào tạo chính quy tại Đại học Bách khoa Hà Nội (sau đây viết tắt là ĐHBK Hà Nội), căn cứ các quy định liên quan tại Nghị định số 84/2020/NĐ -CP ngày 17 /7/2020 của Chính phủ và Quy chế Quản lý tài chính của'}}, {'id': 453466849343023303, 'distance': 0.6189201474189758, 'entity': {'metadata': {'source': '20230710 1. QĐ Học bổng KKHT 2023.pdf', 'page': 0, 'id

In [116]:
docs = list((res["entity"]["text"], res["entity"]["metadata"]["source"],res["distance"]) for res in results[0])  # docs = [(text:str, source:str, distance:float), ...]
from langchain.schema import Document

# Chuyển docs thành danh sách các Document của LangChain
relevant_docs  = [
    Document(page_content=text, metadata={"source": source, "distance": distance})
    for text, source, distance in docs
]


In [117]:
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    print(f"Source: {doc.metadata['source']}\n")


--- Relevant Documents ---
Document 1:
Hà Nội . 2. Quy định này áp dụng đối với sinh viên đại học hình thức đào tạo chính quy văn bằng thứ nhất của ĐHBK Hà Nội . Điều 2. Các quy định chung 1. Học bổng khuyến khích học tập (KKHT) cấp cho các sinh viên được lựa chọn bởi Hội đồng xét cấp học bổng

Source: 20230710 1. QĐ Học bổng KKHT 2023.pdf

Document 2:
và cấp học bổng cho sinh viên đại học hình thức đào tạo chính quy tại Đại học Bách khoa Hà Nội (sau đây viết tắt là ĐHBK Hà Nội), căn cứ các quy định liên quan tại Nghị định số 84/2020/NĐ -CP ngày 17 /7/2020 của Chính phủ và Quy chế Quản lý tài chính của

Source: 20230710 1. QĐ Học bổng KKHT 2023.pdf

Document 3:
học phí của học kỳ lấy điểm xét cấp học bổng . 4. Học bổng KKHT có 3 mức: a) Học bổng loại khá (loại C): Bằng tổng học phí của tất cả các học phần tính điểm trung bình học kỳ (GPA) . b)Học bổng loại giỏi (loại B): Bằng 1,2 lần mức học bổng loại khá .

Source: 20230710 1. QĐ Học bổng KKHT 2023.pdf

Document 4:
Tài liệu cung cấp 

In [119]:
import os

from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [128]:

# Combine the query and the relevant document contents
combined_input = (
    "Here are some documents that might help answer the question: "
    + query
    + "\n\nRelevant Documents:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nPlease provide an answer based only on the provided documents. If the answer is not found in the documents, respond with 'I'm not sure'."
)

# Create a ChatOpenAI model
model = llm

# Define the messages for the model
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input),
]

# Invoke the model with the combined input
result = model.invoke(messages)

# Display the full result and content only
print("\n--- Generated Response ---")
print("Full result:")
print(result)
# print("Content only:")
# print(result.content)


--- Generated Response ---
Full result:
content='Đại học Bách Khoa Hà Nội cung cấp một loại học bổng gọi là học bổng khuyến khích học tập (KKHT). Dưới đây là các thông tin chính về học bổng này:\n\n### 1. Mức học bổng\nHọc bổng KKHT có ba mức dựa trên điểm GPA và điểm rèn luyện:\n- **Loại A (Xuất sắc)**: GPA ≥ 3,6 và điểm rèn luyện ≥ 90 điểm.\n- **Loại B (Giỏi)**: GPA ≥ 3,2 và điểm rèn luyện ≥ 80 điểm.\n- **Loại C (Khá)**: GPA ≥ 2,5 và điểm rèn luyện ≥ 65 điểm.\n\n### 2. Đối tượng áp dụng\nHọc bổng này áp dụng cho sinh viên đại học hệ chính quy văn bằng thứ nhất tại Đại học Bách Khoa Hà Nội.\n\n### 3. Điều kiện cấp học bổng\n- Sinh viên phải có thành tích học tập và rèn luyện trong học kỳ trước.\n- Quỹ học bổng được hình thành từ 8% học phí của học kỳ xét cấp học bổng.\n- Phải không quá thời gian học tập chuẩn và đáp ứng các tiêu chí liên quan đến chương trình đào tạo.\n\n### 4. Công bố kết quả\nKết quả xét duyệt và danh sách sinh viên được cấp học bổng sẽ được công bố trên cổng thông